In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import gudhi as gd
import json

from glob import glob
import os
import tifffile as tf
import wasserstein as ws

from scipy import ndimage
from sklearn import manifold, cluster, decomposition, preprocessing

In [14]:
def get_diagrams(jsonfiles):
    diagh0 = [ np.empty((0,2)) for i in range(len(jsonfiles)) ]
    diagh1 = [ np.empty((0,2)) for i in range(len(jsonfiles)) ]

    for i in range(len(jsonfiles)):
        with open(jsonfiles[i]) as f:
            diag = [tuple(x) for x in json.load(f)]
        h1mask = np.sum([ diag[j][0] == 1 for j in range(len(diag)) ])
        diagh0[i] = np.asarray( [ x[1] for x in diag[h1mask:] ] )
        if h1mask > 0:
            diagh1[i] = np.asarray( [ x[1] for x in diag[:h1mask] ] )

    return diagh0, diagh1

def bottleneck_matrix(diagh0, diagh1, dfunction=gd.bottleneck_distance, **kwargs):
    bottleneck_h0 = np.zeros( (len(diagh0), len(diagh0)) )
    bottleneck_h1 = np.zeros( (len(diagh0), len(diagh0)) )
    
    for i in range(len(bottleneck_h0) - 1):
        for j in range(i+1, len(bottleneck_h0)):
    
            ## H0 ##
            d = dfunction(diagh0[i], diagh0[j], **kwargs)
            bottleneck_h0[i,j] = d
            bottleneck_h0[j,i] = d
    
            ## H1 ##
            d = dfunction(diagh1[i], diagh1[j], **kwargs)
            bottleneck_h1[i,j] = d
            bottleneck_h1[j,i] = d

    return bottleneck_h0, bottleneck_h1

def save_dmatrix(mtrx, filename):
    N = len(mtrx)
    dflat = mtrx[np.triu_indices(N, k=1)]
    pd.Series(dflat).to_csv(filename, index=False, header=None)
    print(filename)

    return dflat

In [16]:
gsrc = '../gd_trans/'
sample = 'D2'
tsrc = gsrc + sample + '/'
ws_order = 1
dst = tsrc

transcriptomes = sorted([foo.split('/')[-2] for foo in glob(tsrc + '*/')])
print(len(transcriptomes), 'transcriptomes')
level = 'super'

97 transcriptomes


In [17]:
tidx = 31
print('Checking', transcriptomes[tidx])
jsonfiles = sorted(glob(tsrc + transcriptomes[tidx] + '/*' + level + 'level.json'))
diag0, diag1 = get_diagrams(jsonfiles)

if len(jsonfiles) == 0:
    print('\n****\nNo JSONs detected for',transcriptomes[tidx], '\t[{}]\n****\n'.format(tidx))
else:
    print('Found',len(jsonfiles),'cells')

Checking GLYMA_06G090100
Found 135 cells


In [18]:
# 1-Wasserstein

h0, h1 = bottleneck_matrix(diag0, diag1, ws.wasserstein_distance, order=ws_order, keep_essential_parts=False)
dmatrix = h0 + h1
filename = tsrc + transcriptomes[tidx] + '_-_' + level + 'level_wasserstein{}.csv'.format(ws_order)
_ = save_dmatrix(dmatrix, filename)

../gd_trans/D2/GLYMA_06G090100_-_superlevel_wasserstein1.csv


In [19]:
# Bottleneck

h0, h1 = bottleneck_matrix(diag0, diag1, gd.bottleneck_distance)
dmatrix = np.maximum(h0, h1)
filename = tsrc + transcriptomes[tidx] + '_-_' + level + 'level_bottleneck.csv'
dflat = save_dmatrix(dmatrix, filename)

../gd_trans/D2/GLYMA_06G090100_-_superlevel_bottleneck.csv


In [13]:
N = len(jsonfiles)
A = np.zeros((N,N))
A[np.triu_indices(N,k=1)] = dflat
A += A.T
A.shape

(135, 135)

In [47]:
bdh0, bdh1 = bottleneck_matrix(diag0, diag1, gd.bottleneck_distance)
bd_mtrx = np.maximum(bdh0, bdh1)
dflat = bd_mtrx[np.triu_indices(len(jsonfiles), k=1)]
filename = tsrc + transcriptomes[tidx] + '_-_' + level + 'level_bottleneck.csv'
pd.Series(dflat).to_csv(filename, index=False, header=None)

array([28.5, 30. , 14. , 32. , 34.5, 28.5, 32. , 21. , 34.5, 32. ])

In [8]:
pd.read_csv(filename, header=None)

,0
0,15.0
1,26.0
2,55.0
3,25.0
4,29.0
...,...
9040,56.0
9041,21.0
9042,23.5
9043,30.0
